In [30]:
import nltk
from nltk.tokenize import word_tokenize
import time, random
from threading import Timer
import re
import string
from nltk.sentiment import SentimentIntensityAnalyzer
import emoji
#nltk.download('vader_lexicon')

In [31]:
######
###ELIZA is an AI chatbot developed by Dr. Joseph Weizenbaum in 1966 in the AI lab at MIT
### It was created to converse in natural language by a computer. It is considered robo-pyschotherapist,
### a machine acts as Doctor, making conversation with humans through interactive chat
###
###This motto of this assignment is creating a chatbot using Python that is similar to Eliza, 
### making interactive conversation with the users.
###Some of the main features are as follows:
###-- Greet the user initially and asks for his/her name
###-- uses his/her name throught the conversation just like human-beings in natural language
###-- Identifies the context of the conversation and responds appropriately
###-- Greet the users at the end of the conversation
#
###Detailed features:
###-- Transforms first/second person pronouns to make meaningful conversation
###-- Identifies main context of the conversation by which problem of the user is identified and suggests reasonable solutions
#
###Additional features:
###-- Identifies if the user uses curse/swear/bad words in his/her conversation and if any inappropriate content is found, it terminates the conversation
###-- Uses EMOJIs or EMOTICONs to make the conversation more interactive wherever it is appropriate
###-- If user is idle for certain amount of time, it reminds the user in a polite language whether he/she needs any help
###-- If the user just enters blank messages, it helps the user understand how it works in natural language
###-- Sentiment Analysis of user inputs and suggestions
#
###Algorithm:
###1. Asks the user to enter his/her name
###2. Fetch the name entered by the user. If user enters nothing, the default user name will be considered 'User'
###3. When prompts for user name, if user remains idle for 50 seconds, it will remind the user to enter his/her name again
###4. If the user enters some curse/bad words when asked for name, the conversation will be ended
###5. Once, the user name is identified, it would ask what the user is looking for. It chooses a random response from the available responses to initiate the 
###   actual conversation. Also, it uses the username throughout the conevrsation to make it more lively and natural
###6. A pool of patterns and responses are pre-defined. Depending upon the input provided by the user, it matches the patterns and 
###   responds to the user according to the context
###7. If it identifies any first/second person pronouns, it makes appropriate transformations while responding
###8. If the responses contain any text related to EMOJIs or EMOTICONs, it converts that text to EMOJIs using 'emoji' package in python
###9. If the respinses contain curse/swear/bad words, it ends the conversation immediately with apt message
##10. If user input contains any End conversation patterns like Bye, End, Quit, Exit, it ends the chat by responding to the user 

###Execution Flow:
### 1.Starts at main()
### 2.Invokes fetch_username() to identify the username and pass it as parameter to start_conv()
### 3. Performs profanity check using verify_profanity() and fetch user name by identifying the patterns using find_specific_patterns() 
###    ---- Patterns to identify name   => ["I'm", "name","call me", "it's"]
###    ---- Trigger the timer if user is idle for 50secs and prompts a suitable message to remind him in the methods enter_text() and enter_name()
###          --- enter_text() : during conversation    --- enter_name() : initial greeting when asked for user name
### 4. Invokes interpret_userinput() to identify the context of the user conversation using Regex patterns in the response_pool dictionary
### 5. Transforms First/Second person pronouns using firstPronoun_to_second_form() and first_pronouns_pool dictionary
### 6. Ends the conversation when end/quit patterns are identifed. Patterns used to end: 
###    ---- end_conv_words = [r'.*(bye).*', r'.*(quit).*', r'.*(exit).*',  r'.*(end).*']   

### Sample conversation:
###
### [Eliza] Hi. I am a psychotherapist. What is your name?
### [User]  I'm Gayathri
### [Eliza] Hi Gayathri. Please enter your request 
### [Gayathri]  I feel so dull today
### [Eliza] What made that you feel so dull today  ?
### [Gayathri]  Maybe I am sad
### [Eliza] Don't worry. Tell me all of your concerns
### [Gayathri]  I need some rest
### [Eliza] Have a good sleep. 🌃
### [Gayathri]  Hello! are you there?
### [Eliza] Hello! Please tell me. 🤝
### [Gayathri]  I feel hungry
### [Eliza] You came to me because you feel hungry  ?
### [Gayathri]  yes, I am hungry and want to eat something
### [Eliza] Eat healthy.
### [Gayathri]  I don't have control on my appetite
### [Eliza] I suggest you to stay away from eating disorders. Try to eat healthy.
### [Gayathri]  I am angry because of long working hours.
### [Eliza] Do you believe it is normal to be you are angry because of long working hours  ?
### [Gayathri]  Maybe
### [Eliza] I understand, please tell me more.
### [Gayathri]  bye
### [Eliza] Bye. Have a nice day! 🤝

#************************************************************************
### Sample sentiment analysis of the user inputs at the end of conversation
##
##  *******************************************************************8
##
### [Gayathri]  I want to read
### [Eliza] Interesting! please tell me more.
### [Gayathri]  I want to read to get rid of bore
### [Eliza] Try to focus.
### [Gayathri]  sure
### [Eliza] I understand, please tell me more.
### [Gayathri]  Bye for now.
### [Eliza] Bye. Have a nice day! 🤝
#   *****************************************************************
### Overall Negative Sentiment % in your input: 24.585714285714285
### Overall Positive Sentiment % in your input: 36.2
### Your overall positive sentiment score seems high. Keep it up!



### References:

# https://www.artofmanliness.com/articles/how-to-comfort-someone-whos-sadcrying/
# https://www.thehrdirector.com/features/hr-in-business/seven-steps-to-calm-an-angry-person-down-in-ten-minutes/
# https://www.huffpost.com/entry/what-to-say-anxiety_n_5978918
# https://7esl.com/ways-to-say-good-night/
# https://stackoverflow.com/questions/38240963/python-check-if-any-of-the-words-in-a-list-are-present-in-a-document
# http://www.unicode.org/emoji/charts/full-emoji-list.html#1f347
# https://www.grammar-monster.com/glossary/first_person.htm
# https://www.youtube.com/watch?v=k8SXsT5TLxQ
# https://www.computerhope.com/jargon/e/eliza.htm
# https://www.geeksforgeeks.org/python-sentiment-analysis-using-vader/

In [32]:
#this method is used to fecth the name of the user
def fetch_username():       
    print("[Eliza] Hi. I am a psychotherapist. What is your name?", end = '\n')
    #Assign deafult user's name 
    default_user = "User"                                  
    name_input = enter_name()
    get_words = name_input.split()      
    name_response_words = ["I'm", "name","call me", "it's"]
    end_response_words = ["bye", "end", "exit", "quit"]
    identified_patterns = []
    identified_end_patterns = []
    #Check if user has entered any bad words when asked for name
    if not verify_profanity(name_input):
        identified_name_patterns = find_specific_patterns(name_response_words,name_input)
        identified_end_patterns = find_specific_patterns(end_response_words,name_input)

        if identified_name_patterns and len(identified_end_patterns) == 0:
            #if some common words are identified in input and no end conversation words found, mostly last word will be the name
            user_name = get_words[-1] 
        elif identified_end_patterns:
            #if user enters words like "Bye, Quit, End, Exit" when asked for name, user name will be considered "User"
            user_name = default_user
        elif name_input:
            #if some common words are not identified in input, user might enter just name
            user_name = name_input
        else:
            #if user does n't enter anything, assign "User" as user name
            user_name = default_user
        return user_name
    else:
        user_name = "Inappropriate content"
        return user_name

In [33]:
#this method is used to identify given set of patterns in the text
#parameter 1: keyword_list => patterns/words you want to identify in the text
#parameter 2: The text content in which you want to search for patterns
def find_specific_patterns(keyword_list, input_text):
    identified_patterns = []
    for word in keyword_list:
        pattern=re.compile(word, re.IGNORECASE)
        result=pattern.search(input_text)
        if result:
            identified_patterns.append(result)
    return identified_patterns
        

In [34]:
#Once the user name is identified, ELIZA starts conversation starts here
def start_conv(user_name):
    prompt_sentences = ["Please enter your request ","How can I help you today?", "What do you want to know?", "What would you like to discuss?"]
    propt_sent = prompt_sentences[random.randint(0,2)]
    end_conv_words = [r'.*(bye).*', r'.*(quit).*', r'.*(exit).*',  r'.*(end).*']                           
    end_conversation_patterns = []
    if user_name == "Inappropriate content":
        print('Inappropriate input. Ending converstaion')
        return
    else:
        print("[Eliza] Hi %s. %s" %(user_name,propt_sent), end = '\n')
        while True:
            user_input = enter_text(user_name) 
            end_conversation_patterns = find_specific_patterns(end_conv_words,user_input)

            #check if there are any bad words in the text entered by the user
            #If found, end the conversation with appropraite message
            if(verify_profanity(user_input)):
                print("Inappropriate input. Ending converstaion")
                break
             #If no bad words found, proceed to next stage
            else:
                #check if there are any quit patterns like bye, end, quit, exit
                #If no quit patterns found, proceed to look for appropriate response from the response pool
                if not end_conversation_patterns:
                    response = interpret_userinput(user_input) 
                    #if response is found, print the response to the user
                    if response:
                        print("[Eliza] " + response, end = '\n')
                    #if no, print some message to continue the conversation
                    else:
                        print("[Eliza] Please tell me more", end = '\n')
                #if any quit patterns found, end the conversation with appropriate message
                else: 
                    print(emoji.emojize("[Eliza] Bye. Have a nice day! :handshake:"))
                    break

In [35]:
#This method is used to ask the name of the use if he/she is idle for first 50 seconds
def enter_name():
    #set the idle time as 50 seconds
    timeout = 50
    #create the timer
    tmr = Timer(timeout, print, ['\n[Eliza] Hi. Are you still there? Could you please tell me your name?\n[User] ' ] )
    #start the timer
    tmr.start()
    #If he/she enters any input, identify the name from the input, else prompt the message asking the user to enter name
    name = input('[User] ') 
    #stop the timer
    tmr.cancel()
    return name

In [36]:
#This method is used when the user stays idle for minimum of 50 secs in the middle of the conversation
def enter_text(username):
    timeout =50
    tm = Timer(timeout, print, ['\n[Eliza] Hi %s. Are you still there? I am waiting for your input... ' %(username)]) 
    tm.start()
    input_text = input('[%s] ' %(username)) 
    tm.cancel()
    return input_text

In [37]:
#This method is used to summarize the overall sentiment of the user inputs
def analyze_sentiment(userInput):
    intensity_analyze = SentimentIntensityAnalyzer()
    score = intensity_analyze.polarity_scores(userInput)
    negative_scores.append(score['neg'])
    positive_scores.append(score['pos'])


In [38]:
#This method is used to converse with the user with appropriate responses
def interpret_userinput(user_input):
    #analyze sentiment of user input
    analyze_sentiment(user_input)
    #remove punctuations from input text
    user_input = user_input.translate(str.maketrans('','',string.punctuation))

    #Iterate over the response pool to look for matched patterns and responses
    for pattern, responses in response_pool.items():   
        match = re.search(pattern, user_input, re.IGNORECASE)
        # if matched with any of the responses, 
        if match: 
            matchedText = match.groups()[0]
            #pick a random response from the available responses
            response = random.choice(responses)
            #If the response contains '\2' in it, convert the First person pronouns to second person pronouns to respond           
            if '\2' in response:
                response = firstPronoun_to_second_form(response, user_input, matchedText)
                response = response+' ?'
            return response

In [39]:
#This method is used to transform First person pronoun to second and vice versa
def firstPronoun_to_second_form(responseFromPool, user_input, matchedText):
    responseFromPool = re.sub('\2','',responseFromPool)
    sub_response = ''
    
    for word in user_input.lower().split():
        #Look for the first/second pronouns in the pronoun pool
        #If found, replace it
        if word in first_pronouns_pool.keys():
            second = first_pronouns_pool[word]
            sub_response += second+' '
        else:
        #else append the word directly
            sub_response += word+' '
            
    return responseFromPool + ''+sub_response
    

In [40]:
#This method is used to verify for bad words in the user inpu
def verify_profanity(user_input):
    #In this scenario, I just added only few patterns that are considered curse/swear words
    #To demonstrate the purpose, I added bad and verybad as examples
    bad_words = [r'.*(bad).*',r'.*(verybad).*', r'\b(F|f)[A-z]{2}(K|k)',r'\b(S|s)[A-z]{2}t',r'\b(A|a)[A-z]{1}(S|s)']
    inappropriate_patterns =[]

    for word in bad_words:
        pattern=re.compile(word, re.IGNORECASE)
        result=pattern.search(user_input)
        if result:
            inappropriate_patterns.append(result)
            return inappropriate_patterns
    

In [41]:
#First/Second Person propnoun pool used while giving responses by Eliza
first_pronouns_pool = {
    "i":"you",
    "me":"you",
    "mine":"yours",
    "my":"your",
    "we":"you",
    "us":"you",
    "ours": "yours",
    "our": "your",
    "am":"are",
    "yours":"mine",
    "you":"me",
    "your":"my"
}

In [42]:
#Eliza will look for certain patterns from this pool and choose one random response from the available responses
response_pool = {  
   
# \2 will be replaced by second/first person pronoun whichever is apt
r'^i (am|like|feel|hate) (.+)':   
    [
    "You came to me because \2",
    "You are here because \2",
    "How often \2",
    "Do you believe it is normal to be \2",
    "What made that \2",
    "Tell me more about what made that \2"
    ],

    
r'(.*)(sad|depressed|unhappy| not well | stress | pain | worry | unhealthy)(.*)' :
    [ 
    "It sounds like you're worried about something. What are you worried about?",
    "Tell me everything that’s bothering you",
    "Tell me everything that’s worrying you",
    "Don't worry. Tell me all of your concerns",
    "Help me understand more about what you’re feeling",
    "What set off these feelings?",
    "What’s the thing that’s worrying you the most?",
    "What’s the worst that could happen?",
    "I'm sorry you're going through that."
    ],

r"(\b)(hi|hello|how are you|hey| what's up| how you doing)(.*)":
    [
    emoji.emojize("Hello! Please tell me. :handshake:"),
    emoji.emojize("Hi! How can I help? :handshake:"),
    emoji.emojize("Hi! How are you doing today? :handshake:")
    ],
r'(.*)(eat|hungry|appetite)(.*)':
    [
        "You deserve to be happy",
        "Eat healthy.",
        "I suggest you to stay away from eating disorders. Try to eat healthy.",
        "Do you know that our eating habits relfect our mental health?"
    ],
r'(.*)(die|kill|suicide)(.*)':
    [
    "What leads you to think that? Did you consult your family regarding this?",
    "Help me understand why you decided that.",
    "Did you speak with your friends about your concerns?",
    "Try to look at the positive side of your life."
    ],

r'(.*)(inactive|uninterest |bore|dull)(.*)':
    [
    "Is there any way to improve the situation?",
    "Try something else that interests you.",
    "Try to improve.",
    "Try to focus.",
    "Try to find alternatives.",
    "Try to engage yourself in the things that interest you"
    ],

r'(.*)(angry | furious | hate | frustrate)(.*)':
    [
    "It's not good to indulge yourself in anger.",
    "I need you to stay calm for a moment.",
    "Hmmm? What made you angry?",
    "Hmm, I can understand how frustrated you are. ",
    "I understand.Try to stay calm for a while",
    "What's that about?"
    ],

r'(.*)(happy| joy|fun |active |hope| interest |curious)(.*)':
    [
    "It's good to hear that something makes you happy.",
    "Glad to hear that you found fun",
    "Great! you found some joyful things",
    "Do things that make you happy.",
    "Seems like you're interested in something. Tell me more.",
    "Looks like you're curious in nature"
    ],

r'(.*)(fear| phobia| frighten|scare|danger|inferior)(.*)':
    [
    "I'm sorry you're going through that.",
    "Try to overcome it with your friends' help",
    "Have you talked about your fears with your family?",
    "Try to talk to friends who can understand you",
    "This is not your fault.",
    "That must be really hard for you."
    ],

r'(.*)(not productive| lazy |inefficient |unable |less | help)(.*)':
    [ "Try to put some extra efforts.",
    "Try to stay active",
    "Engage yourself in the things you interested more.",
    "Sometimes it's okay. Do your best.",
    "Learn and grow."
    ],
r'(.*)(mother|father|brother|sister|cousin|family|friends)(.*)':
    [
        "Tell me more about your family or friends.",
        "With whom else would you like to spend your time?"
    ],

r'(.*)(because|since|due to)(.*)':
    [  
    "Are you sure that it is because of that?",
    "Do you think it is the only reason?",
    "Are there any additional reasons?",
    "Is it one of the important reasons you want to consider?"
    ],
r'(\b)(y(es|ep|up))(\b)':
    [
     "You seem quite confident saying Yes",
     "I understand. Tell me more.",
     "Are you sure?"
    ], 
                
r'(\b)no(\b)':
    [
      "Are you sure?",
      "Are you sure about your NO?"
    ],
    
r'(\b)(sleep|tired|rest|^good night)(.*)':
   [
       emoji.emojize("Nighty Night! :sleeping_face:"),                     
       emoji.emojize("Goodnight! :sleeping_face:"),
       emoji.emojize("Sleep well. :sleeping_face:"),
       emoji.emojize("Sleep tight! :sleeping_face:"),
       emoji.emojize("Have a good sleep. :night_with_stars:"),
       emoji.emojize("Night Night. :sleeping_face:")
   ],
    
               
r'(.*)\?':
    [
     "Why do you ask that?",
     "You seem curious"
    ],
r'(^(?![\s\S]))':
    [
        "I can understand only if you enter something.",
        "Please enter something so that I can help you.",
        "I know it is difficult to speak sometimes. But, I can understand only if you enter something."
    ],    
r'(.*)':
    [
      "Please tell me how can I help you?",        
      "I understand, please tell me more.",
      "Interesting! please tell me more.",
      "Sorry, I did n't understand. Please tell me more."
    ]

}

In [43]:
def main():
    start_conv(fetch_username())      
    
if __name__ == "__main__":
    negative_scores = []
    positive_scores = []
    
    try:
        main()
    except:
        print("Exception occurred")
    finally:
        if negative_scores and positive_scores:
            neg_percent = (sum(negative_scores)/len(negative_scores))*100
            pos_percent = (sum(positive_scores)/len(positive_scores))*100
            print("Overall Negative Sentiment % in your input:",neg_percent)
            print("Overall Positive Sentiment % in your input:",pos_percent)
            if(neg_percent>pos_percent):
                print('Your overall negativity in your words is more. Try to find activities that are good and makes you happy')
            else:
                print('Your overall positive sentiment score seems high. Keep it up!')
    

[Eliza] Hi. I am a psychotherapist. What is your name?


[User]  Gayathri


[Eliza] Hi Gayathri. Please enter your request 


[Gayathri]  Bye


[Eliza] Bye. Have a nice day! 🤝
